In [1]:
import pandas as pd
import numpy as np

### Amendments with Federal Budget

In [2]:
amendments = pd.read_csv('../data/2016-12-22-amendments.xz', 
                         dtype={'proposal_id': np.str,
                               'amendment_beneficiary': np.str,
                               'amendment_program_code': np.str,
                               'amendment_proposal_tranfer_value': np.float,
                               'amendment_tranfer_value': np.float})
amendments.fillna('', inplace=True)
amendments.shape

(81169, 10)

In [3]:
amendments.iloc[0]

proposal_id                                                      588275
proponent_qualification             parliamentary amendment beneficiary
amendment_program_code                                    5100020120077
amendment_number                                               31250010
congressperson_name                                    ANTONIO BALHMANN
amendment_beneficiary                                    07535446000160
tax_indicative                                                       no
congressperson_type                                          individual
amendment_proposal_tranfer_value                                 292500
amendment_tranfer_value                                          292500
Name: 0, dtype: object

### Agreements made with Federal Budget

In [4]:
agreements = pd.read_csv('../data/2016-12-22-agreements.xz', 
                         usecols=(['agreement_number', 'proposal_id', 
                                   'agreement_end_date','date_signed', 'situation']),
                         dtype={'agreement_number': np.str, 
                                'proposal_id': np.str,
                                'situation': np.str}, 
                         parse_dates=['agreement_end_date', 'date_signed'],
                         low_memory=False)
agreements.fillna('', inplace=True)
agreements.shape

(105276, 5)

In [5]:
agreements.iloc[0]

agreement_number                                        782624
proposal_id                                             716232
date_signed                                2014-01-01 00:00:00
situation             Prestação de Contas enviada para Análise
agreement_end_date                         2014-01-09 00:00:00
Name: 0, dtype: object

### Agreements related to amendments

In [6]:
agreements_with_amendments = agreements.merge(amendments, on='proposal_id')
agreements_with_amendments = agreements_with_amendments.filter(['amendment_number', 
                                                                'congressperson_name', 
                                                                'amendment_beneficiary',
                                                                'date_signed', 
                                                                'agreement_end_date',
                                                                'agreement_number', 
                                                                'situation'])
agreements_with_amendments.shape

(44072, 7)

In [7]:
agreements_with_amendments.iloc[0]

amendment_number                                                  81001455
congressperson_name      COMISSAO MISTA DE PLANOS, ORCAMENTOS PUBLICOS ...
amendment_beneficiary                                       11354180000126
date_signed                                            2016-01-02 00:00:00
agreement_end_date                                     2018-09-29 00:00:00
agreement_number                                                    818977
situation                                                      Em execução
Name: 0, dtype: object

---
## Impeded Non-Profit Entities - CEPIM

This dataset gather Non-profit entities that are prevented from entering into agreements, onlending agreements or terms of partnership with the federal public administration.

Origin of the information: Controladoria-Geral da União - CGU (Comptroller General of the Union)

In [8]:
impeded_non_profit_entities = pd.read_csv('../data/2016-12-20-impeded-non-profit-entities.xz', 
                                              dtype={'company_cnpj': np.str,
                                                     'agreement_number': np.str})
impeded_non_profit_entities.shape

(4591, 5)

In [9]:
impeded_non_profit_entities.iloc[0]

company_cnpj                                            00002444000166
compay_name          SINDICATO REGIONAL DOS TRABALHADORES E TRABALH...
agreement_number                                                629097
grating_body                     MINISTERIO DO DESENVOLVIMENTO AGRARIO
impediment_reason              INEXECUÇÃO DO OBJETO (TOTAL OU PARCIAL)
Name: 0, dtype: object


First we need to get the agreements in which entities were impeded:

In [10]:
impeded_entities_w_start_date = agreements_with_amendments.merge(
                                    impeded_non_profit_entities, 
                                    left_on=(['amendment_beneficiary', 
                                              'agreement_number']), 
                                    right_on=(['company_cnpj', 
                                               'agreement_number']))

impeded_entities_w_start_date = impeded_entities_w_start_date.filter(['company_cnpj',
                                                                     'compay_name',
                                                                     'agreement_number',
                                                                     'agreement_end_date',
                                                                     'grating_body',
                                                                     'impediment_reason'])

impeded_entities_w_start_date = impeded_entities_w_start_date.rename(columns = 
                                             {'agreement_end_date':'date_impended', 
                                              'agreement_number': 'impended_agreement'})
impeded_entities_w_start_date.iloc[0]

company_cnpj                                          08366579000112
compay_name           INSTITUTO CULTURAL AFRO-BRASILEIRO OLUFON DEYI
impended_agreement                                            748628
date_impended                                    2011-07-06 00:00:00
grating_body                                PRESIDENCIA DA REPUBLICA
impediment_reason                            MOTIVO NÃO ESPECIFICADO
Name: 0, dtype: object

Because the dataset doesn't gives the date when the entity becomes impended, we are using the end date of the agreement where the entity was impended as a minimum date called here as **date_impended**.

So **date_impended** means that we are concerned only with agreements signed after this date.

In [11]:
agreements_after_impended = agreements_with_amendments.merge(
                                        impeded_entities_w_start_date, 
                                        left_on=(['amendment_beneficiary']), 
                                        right_on=(['company_cnpj']))

#### Querying the agreements signed after the entities were impended

Below we have a list of agreements that are still in execution and are related to the amendments that have as beneficiaries non-profit entities that are impeded. In addition, the difference between the date of signature of the agreements in execution and the date of entities disability is less than 2 years.

In [12]:
agreements_after_impended.query('situation == \'Em execução\' and \
                                 date_impended < date_signed and \
                                 date_signed.dt.year - date_impended.dt.year < 2').shape

(7, 13)

In [13]:
agreements_after_impended.query('situation == \'Em execução\' and \
                                 date_impended < date_signed and \
                                 date_signed.dt.year - date_impended.dt.year < 2')

,amendment_number,congressperson_name,amendment_beneficiary,date_signed,agreement_end_date,agreement_number,situation,company_cnpj,compay_name,impended_agreement,date_impended,grating_body,impediment_reason
6,36500017,OSMAR SERRAGLIO,76575604000128,2011-12-29,2017-01-09,756696,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
7,33090023,NELSON MEURER,76575604000128,2011-12-29,2017-04-16,757799,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
8,24050011,LUIZ CARLOS SETIM,76575604000128,2011-12-29,2017-04-16,757828,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
9,10560010,SERGIO SOUZA,76575604000128,2011-12-29,2017-05-15,757915,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
10,33320012,RICARDO BARROS,76575604000128,2011-12-29,2017-05-20,758001,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
11,20380001,FLAVIO JOSE ARNS,76575604000128,2011-12-29,2017-04-16,758030,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO
12,31760009,DILCEU SPERAFICO,76575604000128,2011-12-29,2017-03-31,760953,Em execução,76575604000128,SOCIEDADE EVANGELICA BENEFICENTE DE CURITIBA,747398,2011-09-11,MINISTERIO DO TURISMO,MOTIVO NÃO ESPECIFICADO


---
## Inident and Suspended Companies - CEIS

This dataset gather companies and individuals who have suffered sanctions by the organs and entities of the public administration of the various federative spheres.

Origin of the information: Controladoria-Geral da União - CGU (Comptroller General of the Union)

In [14]:
inident_and_suspended_companies = pd.read_csv('../data/2016-12-21-inident-and-suspended-companies.xz',
                                              dtype={'sanctioned_cnpj_cpf': np.str,
                                                     'process_number': np.str},
                                              parse_dates = ['sanction_start_date',
                                                             'sanction_end_date', 
                                                             'data_source_date',
                                                             'published_date'], 
                                              low_memory=False)
inident_and_suspended_companies.fillna('', inplace=True)
inident_and_suspended_companies['sanction_start_date'] = pd.to_datetime(
                            inident_and_suspended_companies['sanction_start_date'], 
                            format='%Y-%m-%d')
inident_and_suspended_companies['sanction_end_date'] = pd.to_datetime(
                            inident_and_suspended_companies['sanction_end_date'],
                            format='%Y-%m-%d')
inident_and_suspended_companies.shape

(13084, 17)

In [15]:
inident_and_suspended_companies.iloc[0]

entity_type                                                                            Física
sanctioned_cnpj_cpf                                                                1540142817
name_given_by_sanctioning_body                                                    01540142817
company_name_receita_database                                               FAIAD HABIB ZAKIR
trading_name_receita_database                                                                
process_number                                                           00001655520138260240
sanction_type                                                  Proibição - Lei de Improbidade
sanction_start_date                                                       2014-08-09 00:00:00
sanction_end_date                                                         2019-08-09 00:00:00
sanctioning_body                            Tribunal de Justiça do Estado de São Paulo / 1...
state_of_sanctioning_body                                   

In [16]:
agreements_with_suspended_companies = agreements_with_amendments.merge(
                                        inident_and_suspended_companies, 
                                        left_on='amendment_beneficiary', 
                                        right_on='sanctioned_cnpj_cpf')
agreements_with_suspended_companies.shape

(24, 24)

#### Querying the agreements still running after entities were suspended

Below we have a list of agreements that are still in execution and which the signed date are between sanction's start and end date.

In [17]:
agreements_with_suspended_companies.query('entity_type == \'Juridica\' and \
                                           situation == \'Em execução\' and \
                                           sanction_start_date <= date_signed and \
                                           date_signed <= sanction_end_date'
                                         ).shape

(6, 24)

In [18]:
agreements_with_suspended_companies = agreements_with_suspended_companies.rename(
                                        columns = {'date_signed':'agreement_date_signed'})

In [19]:
agreements_with_suspended_companies.query('entity_type == \'Juridica\' and \
                                          situation == \'Em execução\' and \
                                          sanction_start_date <= agreement_date_signed and \
                                          agreement_date_signed <= sanction_end_date'
                                         ).filter(['amendment_number',
                                                   'agreement_number',
                                                   'congressperson_name',
                                                   'sanctioned_cnpj_cpf',
                                                   'name_given_by_sanctioning_body',
                                                   'company_name_receita_database',
                                                   'trading_name_receita_database',
                                                   'process_number',
                                                   'agreement_date_signed',
                                                   'sanction_start_date',
                                                   'sanction_end_date',
                                                   'sanction_type',
                                                   'sanctioning_body',
                                                   'state_of_sanctioning_body',
                                                   'data_source',
                                                   'data_source_date',
                                                   'published_date'])

,amendment_number,agreement_number,congressperson_name,sanctioned_cnpj_cpf,name_given_by_sanctioning_body,company_name_receita_database,trading_name_receita_database,process_number,agreement_date_signed,sanction_start_date,sanction_end_date,sanction_type,sanctioning_body,state_of_sanctioning_body,data_source,data_source_date,published_date
0,28160003,819313,ROBERTO DE LUCENA,67662437000161,Jose Carlos Mendes,MUNICIPIO DE EUCLIDES DA CUNHA PAULISTA,PREFEITURA MUNICIPAL DE EUCLIDES DA CUNHA PAUL...,62701200600070140000000000000,2015-04-11,2010-11-11,2020-11-11,Proibição - Lei de Improbidade,Justiça Estadual,,Conselho Nacional de Justiça,2011-12-31 00:00:00,
1,28070006,803313,MARA GABRILLI,67662437000161,Jose Carlos Mendes,MUNICIPIO DE EUCLIDES DA CUNHA PAULISTA,PREFEITURA MUNICIPAL DE EUCLIDES DA CUNHA PAUL...,62701200600070140000000000000,2014-07-16,2010-11-11,2020-11-11,Proibição - Lei de Improbidade,Justiça Estadual,,Conselho Nacional de Justiça,2011-12-31 00:00:00,
2,33600004,831772,SILVIO TORRES,67662437000161,Jose Carlos Mendes,MUNICIPIO DE EUCLIDES DA CUNHA PAULISTA,PREFEITURA MUNICIPAL DE EUCLIDES DA CUNHA PAUL...,62701200600070140000000000000,2016-05-17,2010-11-11,2020-11-11,Proibição - Lei de Improbidade,Justiça Estadual,,Conselho Nacional de Justiça,2011-12-31 00:00:00,
4,25410001,765683,CANDIDO VACCAREZZA,67662437000161,Jose Carlos Mendes,MUNICIPIO DE EUCLIDES DA CUNHA PAULISTA,PREFEITURA MUNICIPAL DE EUCLIDES DA CUNHA PAUL...,62701200600070140000000000000,2011-12-30,2010-11-11,2020-11-11,Proibição - Lei de Improbidade,Justiça Estadual,,Conselho Nacional de Justiça,2011-12-31 00:00:00,
5,18180001,781641,NELSON MARQUEZELLI,67662437000161,Jose Carlos Mendes,MUNICIPIO DE EUCLIDES DA CUNHA PAULISTA,PREFEITURA MUNICIPAL DE EUCLIDES DA CUNHA PAUL...,62701200600070140000000000000,2012-12-31,2010-11-11,2020-11-11,Proibição - Lei de Improbidade,Justiça Estadual,,Conselho Nacional de Justiça,2011-12-31 00:00:00,
12,25310017,829062,PAULO MALUF,45131885000104,QUALITY SERVICE LIMPEZA E CONSERVAçãO,MUNICIPIO DE JALES,JALES GABINETE PREFEITO,00073084619988260297,2016-11-24,2015-09-24,2020-09-24,Proibição - Lei de Improbidade,Tribunal de Justiça do Estado de São Paulo / 1...,SP,Conselho Nacional de Justiça,2015-11-17 00:00:00,


---
## National Registry of Punished Companies - CNEP

The National Register of Punished Companies (CNEP) is an information bank maintained by the Federal Comptroller's Office (CGU), whose purpose is to consolidate the relationship of companies that have suffered any of the punishments provided in Law 12,846 / 2013 (Anti-Corruption Law).

Origin of the information: Controladoria-Geral da União - CGU (Comptroller General of the Union)

In [20]:
punished_companies = pd.read_csv('../data/2016-12-21-national-register-punished-companies.xz',
                                              dtype={'sanctioned_cnpj_cpf': np.str,
                                                     'process_number': np.str},
                                              parse_dates = ['sanction_start_date',
                                                             'sanction_end_date', 
                                                             'data_source_date',
                                                             'published_date'], 
                                              low_memory=False)
punished_companies.fillna('', inplace=True)
punished_companies.shape

(9, 17)

In [21]:
len(punished_companies['sanctioned_cnpj_cpf'].unique())

8

As we can see above, this dataset has only 9 records listed and 8 companies. 

In [22]:
punished_companies.iloc[0]

entity_type                                                                Juridica
sanctioned_cnpj_cpf                                                  19823999000105
name_given_by_sanctioning_body    ALIMENTARE SERVIÇOS DE RESTAURANTE E LANCHONET...
company_name_receita_database     ALIMENTARE SERVICOS DE RESTAURANTE E LANCHONET...
trading_name_receita_database                                                      
process_number                                                        001/DFRC/2015
sanction_type                                          Multa - Lei da Empresa Limpa
sanction_start_date                                             2016-08-15 00:00:00
sanction_end_date                                                                  
sanctioning_body                  EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...
state_of_sanctioning_body                                                        DF
data_source                       Empresa Brasileira de Infra-Estrutura Aero

Here we got the agreements related to these punished companies:

In [23]:
agreements_with_punished_companies = agreements_with_amendments.merge(
                                        punished_companies, 
                                        left_on='amendment_beneficiary', 
                                        right_on='sanctioned_cnpj_cpf')
agreements_with_punished_companies.shape

(0, 24)

As of 12/21/2016 there are no agreements related to these punished companies.